# Kaggle team name: vijf5!

# Members:
1. Jesse Coenraad -> JesseCoenraad
2. Jordan van Sintanneland -> JordanvanSint
3. Casper Priem -> CasperPriem
4. Jassar Hasiba -> UrClades

## Imports

In [2]:
import os
import librosa as lr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Audio
import pandas as pd

In [3]:
labels = pd.read_csv('labels_new.csv')

labels.head()

,filename,genre
0,m00248.wav,metal
1,m00230.wav,country
2,m00637.wav,hiphop
3,m00627.wav,metal
4,m00138.wav,reggae


In [4]:
#make a list object of unique genres in labels dataframe
genres = list(labels['genre'].unique())
genres


['metal',
 'country',
 'hiphop',
 'reggae',
 'classical',
 'jazz',
 'rock',
 'pop',
 'blues',
 'disco']

# Feature engineering

In [19]:
from pathlib import Path
import os

In [20]:
# Define the base directory where the subdirectories "cat" and "dog" are located
base_dir = 'Opdracht_3\labeled'

# Initialize lists for audio data and sample frequencies
audio_data = []
sample_freqs = []

# Target length for 5 seconds at a sample rate of 16000 Hz (adjust if different)
target_length = 5 * 16000

# Process each audio file
for label in labels['filename']:
    sub_dir = os.path.join(base_dir, label)
    for file in os.listdir(sub_dir):
        if file.endswith(".wav"):
            file_path = os.path.join(sub_dir, file)
            data, sfreq = lr.load(file_path, sr=None)  # Keep original sample rate

            # Truncate or pad the audio
            if len(data) > target_length:
                # Truncate the data
                data = data[:target_length]
            elif len(data) < target_length:
                # Pad with zeros
                padding = target_length - len(data)
                data = np.pad(data, (0, padding), mode='constant')

            # Append the processed data and sample frequency
            audio_data.append(data)
            sample_freqs.append(sfreq)

# Convert to numpy array
audio_data = np.stack(audio_data, axis=0)
sample_freqs = np.array(sample_freqs)

# Add audio data and sample frequencies to the labels dataframe
labels['audio_data'] = list(audio_data)
labels['sample_freqs'] = sample_freqs

# Create the new dataframe labeled_audio
labeled_audio = labels

# Displaying a sample
print(labeled_audio.head())

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'Opdracht_3\\labeled\\m00248.wav'

# Unsupervised Learning